In [1]:
import pickle as pk
import dgl as d
import pandas as pd
import numpy as np
from datasets.dataset import FeatureLookup
from models.node2vec import Node2vecModel as Node2vec2

In [2]:
#!/usr/bin/env python
__coding__ = "utf-8"
__author__ = "Ng WaiMing"

from numpy import *
from time import sleep


def distEclud(vecA, vecB):
    """
    欧氏距离计算函数
    :param vecA:
    :param vecB:
    :return:
    """
    return sqrt(sum(power(vecA - vecB, 2)))


def randCent(dataMat, k):
    """
    为给定数据集构建一个包含K个随机质心的集合,
    随机质心必须要在整个数据集的边界之内,这可以通过找到数据集每一维的最小和最大值来完成
    然后生成0到1.0之间的随机数并通过取值范围和最小值,以便确保随机点在数据的边界之内
    :param dataMat:
    :param k:
    :return:
    """
    # 获取样本数与特征值
    m, n = shape(dataMat)
    # 初始化质心,创建(k,n)个以零填充的矩阵
    centroids = mat(zeros((k, n)))
    # 循环遍历特征值
    for j in range(n):
        # 计算每一列的最小值
        minJ = min(dataMat[:, j])
        # 计算每一列的范围值
        rangeJ = float(max(dataMat[:, j]) - minJ)
        # 计算每一列的质心,并将值赋给centroids
        centroids[:, j] = mat(minJ + rangeJ * random.rand(k, 1))
    # 返回质心
    return centroids


def kMeans(dataMat, k, distMeas = distEclud, createCent = randCent):
    '''
    创建K个质心,然后将每个店分配到最近的质心,再重新计算质心。
    这个过程重复数次,直到数据点的簇分配结果不再改变为止
    :param dataMat: 数据集
    :param k: 簇的数目
    :param distMeans: 计算距离
    :param createCent: 创建初始质心
    :return:
    '''
    # 获取样本数和特征数
    m, n = shape(dataMat)
    # 初始化一个矩阵来存储每个点的簇分配结果
    # clusterAssment包含两个列:一列记录簇索引值,第二列存储误差(误差是指当前点到簇质心的距离,后面会使用该误差来评价聚类的效果)
    clusterAssment = mat(zeros((m, 2)))
    # 创建质心,随机K个质心
    centroids = createCent(dataMat, k)
    # 初始化标志变量,用于判断迭代是否继续,如果True,则继续迭代

    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        # 遍历所有数据找到距离每个点最近的质心,
        # 可以通过对每个点遍历所有质心并计算点到每个质心的距离来完成
        for i in range(m):
            minDist = inf
            minIndex = -1
            for j in range(k):
                # 计算数据点到质心的距离
                # 计算距离是使用distMeas参数给出的距离公式,默认距离函数是distEclud
                distJI = distMeas(centroids[j, :], dataMat[i, :])
                # 如果距离比minDist(最小距离)还小,更新minDist(最小距离)和最小质心的index(索引)
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            # 如果任一点的簇分配结果发生改变,则更新clusterChanged标志
            if clusterAssment[i, 0] != minIndex:
                clusterChanged = True
            # 更新簇分配结果为最小质心的index(索引),minDist(最小距离)的平方
            clusterAssment[i, :] = minIndex, minDist ** 2

        # print(centroids)

        # 遍历所有质心并更新它们的取值
        for cent in range(k):
            # 通过数据过滤来获得给定簇的所有点
            ptsInClust = dataMat[nonzero(clusterAssment[:, 0].A == cent)[0]]
            # 计算所有点的均值,axis=0表示沿矩阵的列方向进行均值计算
            centroids[cent, :] = mean(ptsInClust, axis=0)

    # 返回所有的类质心与点分配结果
    return centroids, clusterAssment


def biKmeans(dataMat, k, distMeas = distEclud):
    '''
    在给定数据集,所期望的簇数目和距离计算方法的条件下,函数返回聚类结果
    :param dataMat:
    :param k:
    :param distMeas:
    :return:
    '''
    m, n = shape(dataMat)
    # 创建一个矩阵来存储数据集中每个点的簇分配结果及平方误差
    clusterAssment = mat(zeros((m, 2)))

    # 计算整个数据集的质心,并使用一个列表来保留所有的质心
    centroid0 = mean(dataMat, axis = 0).tolist()[0]
    centList = [centroid0]

    # 遍历数据集中所有点来计算每个点到质心的误差值
    for j in range(m):
        clusterAssment[j, 1] = distMeas(mat(centroid0), dataMat[j, :]) ** 2

    # 对簇不停的进行划分,直到得到想要的簇数目为止
    while len(centList) < k:
        # 初始化最小SSE为无穷大,用于比较划分前后的SSE
        lowestSSE = inf
        # 通过考察簇列表中的值来获得当前簇的数目,遍历所有的簇来决定最佳的簇进行划分
        for i in range(len(centList)):
            # 对每一个簇,将该簇中的所有点堪称一个小的数据集
            ptsInCurrCluster = dataMat[nonzero(clusterAssment[:, 0].A == i)[0], :]

            # 将ptsInCurrCluster输入到函数kMeans中进行处理,k=2,
            # kMeans会生成两个质心(簇),同时给出每个簇的误差值
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)

            # 将误差值与剩余数据集的误差之和作为本次划分的误差
            sseSplit = sum(splitClustAss[:, 1])
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:, 0].A != i)[0], 1])
            # print('sseSplit, and notSplit: ', sseSplit, sseNotSplit)

            # 如果本次划分的SSE值最小,则本次划分被保存
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit

        # 找出最好的簇分配结果
        # 调用kmeans函数并且指定簇数为2时,会得到两个编号分别为0和1的结果簇
        bestClustAss[nonzero(bestClustAss[:, 0].A == 1)[0], 0] = len(centList)
        # 更新为最佳质心
        bestClustAss[nonzero(bestClustAss[:, 0].A == 0)[0], 0] = bestCentToSplit

        print(len(centList))
        print('the bestCentToSplit is: ', bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))

        # 更新质心列表
        # 更新原质心list中的第i个质心为使用二分kMeans后bestNewCents的第一个质心
        centList[bestCentToSplit] = bestNewCents[0, :].tolist()[0]

        # 添加bestNewCents的第二个质心
        centList.append(bestNewCents[1, :].tolist()[0])

        # 重新分配最好簇下的数据(质心)以及SSEk
        clusterAssment[nonzero(clusterAssment[:, 0].A == bestCentToSplit)[0], :] = bestClustAss

    return mat(centList), clusterAssment


In [3]:
class a:
    def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
        self.node2vec_dim = node2vec_dim
        self.node2vec_length = node2vec_length
        self.node2vec_walk = node2vec_walk
        self.node2vec_epochs = node2vec_epochs
        self.node2vec_batchsize = node2vec_batchsize
        self.slices = 10

class FeatureLookup:

    def __init__(self):
        self.inner_id_counter = 0
        self.inner_bag = {}
        self.category = set()
        self.category_bags = {}
        self.inverse_map = {}

    def register(self, category, value):
        # 添加进入类别
        self.category.add(category)

        # 如果类别不存在若无则，则新增一个类别子树
        if category not in self.category_bags:
            self.category_bags[category] = {}

        # 如果值不在全局索引中，则创建之，id += 1
        if value not in self.inner_bag:
            self.inner_bag[value] = self.inner_id_counter
            self.inverse_map[self.inner_id_counter] = value
            # 如果值不存在与类别子树，则创建之
            if value not in self.category_bags[category]:
                self.category_bags[category][value] = self.inner_id_counter
            self.inner_id_counter += 1

    def query_id(self, value):
        # 返回索引id
        return self.inner_bag[value]

    def query_value(self, idx):
        # 返回值
        return self.inverse_map[idx]

    def __len__(self):
        return len(self.inner_bag)

def node2vec_embedings(graph, args):

    # config
    p = 1
    q = 0.5
    embedding_dim = args.node2vec_dim
    walk_length = args.node2vec_length
    num_walks = args.node2vec_walk
    epochs = args.node2vec_epochs
    batch_size = args.node2vec_batchsize
    device = 'gpu'

    # config
    trainer = Node2vec2(
        graph,
        embedding_dim = embedding_dim,
        walk_length = walk_length,
        p = p,
        q = q,
        num_walks = num_walks,
        # weight_name = 'w',
        device = device,
    )

    trainer.train(epochs = epochs, batch_size = batch_size, learning_rate = 1e-2)

    node_features = trainer.embedding()

    nodes = node_features.detach().to('cpu').numpy()

    return nodes

# node2vec
def eraser(datasets, args):
    label = None
    userg = d.graph([])
    user_lookup = FeatureLookup()
    ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_table.csv')
    ufile = pd.DataFrame(ufile)
    ulines = ufile.to_numpy()

    for i in range(339):
        user_lookup.register('User', i)

    for ure in ulines[:, 2]:
        user_lookup.register('URE', ure)

    for uas in ulines[:, 4]:
        user_lookup.register('UAS', uas)

    userg.add_nodes(len(user_lookup))

    for line in ulines:
        uid = line[0]
        ure = user_lookup.query_id(line[2])

        if not userg.has_edges_between(uid, ure):
            userg.add_edges(uid, ure)

        uas = user_lookup.query_id(line[4])
        if not userg.has_edges_between(uid, uas):
            userg.add_edges(uid, uas)

    userg = d.add_self_loop(userg)
    userg = d.to_bidirected(userg)

    # node2vec
    ans = None
    ans = np.array(node2vec_embedings(userg, args))
    ans = ans[: 339]

    # 整理特征
    df = np.array(ans)

    pk.dump(df, open(f'./pretrain/user_embeds.pk', 'wb'))

In [4]:
args = a(128, 20, 50, 2000, 32)
# eraser(None, args)

In [5]:
df = pk.load(open(f'./pretrain/user_embeds.pk', 'rb'))
myCentroids, clustAssing = biKmeans(df, 10, distMeas = distEclud)

1
the bestCentToSplit is:  0
the len of bestClustAss is:  339
2
the bestCentToSplit is:  1
the len of bestClustAss is:  330
3
the bestCentToSplit is:  1
the len of bestClustAss is:  311
4
the bestCentToSplit is:  2
the len of bestClustAss is:  19
5
the bestCentToSplit is:  3
the len of bestClustAss is:  9
6
the bestCentToSplit is:  2
the len of bestClustAss is:  15
7
the bestCentToSplit is:  1
the len of bestClustAss is:  302
8
the bestCentToSplit is:  7
the len of bestClustAss is:  7
9
the bestCentToSplit is:  6
the len of bestClustAss is:  12


/home/yxzeng/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/yxzeng/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [6]:
data = np.array(clustAssing[:,0], dtype = 'int32').reshape(-1,)
data = data.tolist()

In [7]:
dic = {}
for i in data:
    if i not in dic:
        dic[i] = 1
    else:
        dic[i] += 1
dic

{1: 295, 4: 4, 0: 9, 7: 4, 8: 3, 2: 3, 6: 6, 5: 6, 3: 3, 9: 6}

In [8]:
ans = 0
for i in dic:
    ans += dic[i]
ans

339

In [9]:
class a:
    def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
        self.node2vec_dim = node2vec_dim
        self.node2vec_length = node2vec_length
        self.node2vec_walk = node2vec_walk
        self.node2vec_epochs = node2vec_epochs
        self.node2vec_batchsize = node2vec_batchsize
        self.slices = 10
        self.part_iter = 100

In [10]:
# 欧氏距离计算


# user-based-parition
def user_based_parition(tensor, args):
    """
        part_iter :: 切分设定
        slices :: 切分片数
    """
    def E_score2(value1, value2):
        return np.sum(np.power(value1 - value2, 2))

    user_embeds = pk.load(open('./pretrain/user_embeds.pk', 'rb'))

    split_Tensor = []

    max_number = 1.2 * user_embeds.shape[0] // args.slices

    import random
    center_id = random.sample(range(0, user_embeds.shape[0]), args.slices)

    center_user_value = []
    for i in range(args.slices):
        center_user_value.append(user_embeds[center_id[i]])

    C = None
    for iterid in range(args.part_iter):
        C = [{} for _ in range(args.slices)]
        Scores = {}
        for userid in range(user_embeds.shape[0]):
            for sliceid in range(args.slices):
                score = E_score2(np.array(user_embeds[userid]), np.array(center_user_value[sliceid]))
                Scores[userid, sliceid] = -score
        Scores = sorted(Scores.items(), key = lambda x : x[1], reverse = True)

        visted = [False for _ in range(user_embeds.shape[0])]
        for i in range(len(Scores)):
            if not visted[Scores[i][0][0]]:
                if len(C[Scores[i][0][1]]) < max_number:
                    C[Scores[i][0][1]][Scores[i][0][0]] = tensor[Scores[i][0][0]]
                    visted[Scores[i][0][0]] = True

        center_user_value_next = []
        for sliceid in range(args.slices):
            temp_user_value = []

            for userid in C[sliceid].keys():
                temp_user_value.append(user_embeds[userid])

            if len(temp_user_value):
                center_user_value_next.append(np.mean(temp_user_value))
            else:
                center_user_value_next.append(0)

        loss = 0.

        for sliceid in range(args.slices):
            score = E_score2(np.array(center_user_value_next[sliceid]), np.array(center_user_value[sliceid]))
            loss += score

        center_user_value = center_user_value_next
        # log(f'iterid {iterid} loss : {loss:.50f}')

    # 开始返回339 * 5825矩阵
    for sliceid in range(args.slices):
        splits_Tensor = np.zeros_like(tensor)

        idx = list(C[sliceid].keys())
        splits_Tensor[idx] = tensor[idx]

        split_Tensor.append(splits_Tensor)  # 1

    return split_Tensor, C

In [11]:
tensor = np.array(pk.load(open('datasets/data/WSDREAM/rt.pk', 'rb')))
args = a(128, 20, 50, 1000, 32)
split_Tensor, C = user_based_parition(tensor, args)

In [12]:
for i in C:
    print(len(i))

40
40
40
33
10
40
40
16
40
40


In [13]:
ans = 0
for i in range(10):
    ans += (np.array(split_Tensor[i]) != 0).sum()
ans

1974675

In [14]:
# 欧氏距离计算


# user-based-parition
def item_based_parition(tensor, args):
    """
        part_iter :: 切分设定
        slices :: 切分片数
    """
    def E_score2(value1, value2):
        return np.sum(np.power(value1 - value2, 2))

    item_embeds = pk.load(open('./pretrain/item_embeds.pk', 'rb'))

    split_Tensor = []

    max_number = 1.2 * item_embeds.shape[0] // args.slices

    import random
    center_id = random.sample(range(0, item_embeds.shape[0]), args.slices)

    center_user_value = []
    for i in range(args.slices):
        center_user_value.append(item_embeds[center_id[i]])

    C = None
    for iterid in range(args.part_iter):
        C = [{} for _ in range(args.slices)]
        Scores = {}
        for userid in range(item_embeds.shape[0]):
            for sliceid in range(args.slices):
                score = E_score2(np.array(item_embeds[userid]), np.array(center_user_value[sliceid]))
                Scores[userid, sliceid] = -score
        Scores = sorted(Scores.items(), key = lambda x : x[1], reverse = True)

        visted = [False for _ in range(item_embeds.shape[0])]
        for i in range(len(Scores)):
            if not visted[Scores[i][0][0]]:
                if len(C[Scores[i][0][1]]) < max_number:
                    C[Scores[i][0][1]][Scores[i][0][0]] = tensor[:,Scores[i][0][0]]
                    visted[Scores[i][0][0]] = True

        center_user_value_next = []
        for sliceid in range(args.slices):
            temp_user_value = []

            for userid in C[sliceid].keys():
                temp_user_value.append(item_embeds[userid])

            if len(temp_user_value):
                center_user_value_next.append(np.mean(temp_user_value))
            else:
                center_user_value_next.append(0)

        loss = 0.

        for sliceid in range(args.slices):
            score = E_score2(np.array(center_user_value_next[sliceid]), np.array(center_user_value[sliceid]))
            loss += score

        center_user_value = center_user_value_next
        # log(f'iterid {iterid} loss : {loss:.50f}')

    # 开始返回339 * 5825矩阵
    for sliceid in range(args.slices):
        splits_Tensor = np.zeros_like(tensor)

        idx = list(C[sliceid].keys())
        splits_Tensor[:,idx] = tensor[:,idx]

        split_Tensor.append(splits_Tensor)  # 1

    return split_Tensor, C

In [15]:
tensor = np.array(pk.load(open('datasets/data/WSDREAM/rt.pk', 'rb')))
args = a(128, 20, 50, 1000, 32)
# split_Tensor, C = item_based_parition(tensor, args)

In [16]:
for i in C:
    print(len(i))

40
40
40
33
10
40
40
16
40
40


In [17]:
ans = 0
for i in range(10):
    ans += (np.array(split_Tensor[i]) != 0).sum()
ans

1974675

In [18]:
from tqdm import *
# RecEarser
def interaction_based_balanced_parition(tensor, args):
    def E_score2(value1, value2):
        return np.sum(np.power(value1 - value2, 2))

    user_embeds = pk.load(open('./pretrain/user_embeds.pk', 'rb'))
    item_embeds = pk.load(open('./pretrain/item_embeds.pk', 'rb'))

    split_Tensor = []

    data = []
    for i in range(tensor.shape[0]):
        for j in range(tensor.shape[1]):
            data.append([i, j])

    max_number = 1.2 * (tensor != 0).sum() // args.slices
    # max_number = 1.2 * Tensor.shape[0] // args.slices
    import random
    center_id = random.sample(data, args.slices)

    # center_id = random.sample(range(0, tensor.shape[0]), args.slices)

    center_user_value = []
    for i in range(args.slices):
        center_user_value.append([user_embeds[center_id[i][0]], item_embeds[center_id[i][1]]])

    C = None
    for iterid in trange(args.part_iter):
        C = [{} for _ in range(args.slices)]
        C_number = [0 for _ in range(args.slices)]
        Scores = {}

        for userid in range(len(data)):
            for sliceid in range(args.slices):
                score_user = E_score2(np.array(user_embeds[data[userid][0]]), np.array(center_user_value[sliceid][0]))
                score_item = E_score2(np.array(item_embeds[data[userid][0]]), np.array(center_user_value[sliceid][1]))
                Scores[userid, sliceid] = - score_user * score_item

        Scores = sorted(Scores.items(), key = lambda x : x[1], reverse = True)

        visted = [False for _ in range(len(data))]
        for i in range(len(Scores)):
            if not visted[Scores[i][0][0]]:
                if C_number[Scores[i][0][1]] < max_number:
                    if data[Scores[i][0][0]][0] not in C[Scores[i][0][1]]:
                        C[Scores[i][0][1]][data[Scores[i][0][0]][0]] = [data[Scores[i][0][0]][1]]  # 把这个item放入对应的user元祖
                    else:
                        C[Scores[i][0][1]][data[Scores[i][0][0]][0]].append(data[Scores[i][0][0]][1])

                    # print(C[Scores[i][0][1]][data[Scores[i][0][0]][0]])

                    visted[Scores[i][0][0]] = True
                    C_number[Scores[i][0][1]] += 1

        center_user_value_next = []

        for sliceid in range(args.slices):
            temp_user_value = []
            temp_item_value = []
            user_mean, item_mean = None, None
            for userid in C[sliceid].keys():
                for itemid in C[sliceid][userid]:
                    temp_user_value.append(user_embeds[userid])
                    temp_item_value.append(item_embeds[itemid])
                    if len(temp_user_value):
                        user_mean = np.mean(temp_user_value)
                    else:
                        user_mean = 0

                    if len(temp_item_value):
                        item_mean = np.mean(temp_item_value)
                    else:
                        item_mean = 0
            center_user_value_next.append([user_mean, item_mean])

        loss = 0.0

        for sliceid in range(args.slices):
            score_user = E_score2(np.array(center_user_value_next[sliceid][0]), np.array(center_user_value[sliceid][0]))
            score_item = E_score2(np.array(center_user_value_next[sliceid][1]), np.array(center_user_value[sliceid][1]))
            loss += (score_user * score_item)

        center_user_value = center_user_value_next
        log(f'iterid {iterid + 1} : loss = {loss:.30f}')
        for sliceid in range(args.slices):
            log(f'C[{sliceid}] number = {len(list(C[sliceid]))}')

    row_idx = [[] for _ in range(args.slices)]
    col_idx = [[] for _ in range(args.slices)]

    for sliceid in range(args.slices):
        temp = np.zeros_like(tensor)

        for userid in C[sliceid].keys():
            row_idx[sliceid] += [userid for _ in range(len(C[sliceid][userid]))]
            col_idx[sliceid] += [itemid for itemid in C[sliceid][userid]]

        temp[row_idx[sliceid], col_idx[sliceid]] = tensor[row_idx[sliceid], col_idx[sliceid]]

        split_Tensor.append(temp)

    return split_Tensor, C

In [20]:
tensor = np.array(pk.load(open('datasets/data/WSDREAM/rt.pk', 'rb')))
args = a(128, 20, 50, 1000, 32)
args.part_iter = 20
split_Tensor, C = interaction_based_balanced_parition(tensor, args)

  0%|          | 0/20 [3:00:52<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
C = np.array(C)
pk.dump(C, file = open('C.pk', 'wb'))